In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,36352
2,Huelva,Confirmados PDIA 14 días,2093
3,Huelva,Tasa PDIA 14 días,"407,85704542354387"
4,Huelva,Confirmados PDIA 7 días,1096
5,Huelva,Tasa PDIA 7 dias,"213,57444901299763"
6,Huelva,Total Confirmados,36551
7,Huelva,Curados,33022
8,Huelva,Fallecidos,395


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  36352.0


/tmp/ipykernel_137813/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  10101.0


/tmp/ipykernel_137813/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_137813/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_137813/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_137813/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 409 personas en los últimos 7 días 

Un positivo PCR cada 194 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,36352.0,1096.0,2093.0,513170.0,213.574449,407.857045,200.0
Huelva-Costa,21282.0,728.0,1456.0,289548.0,251.426361,502.852722,121.0
Condado-Campiña,11552.0,259.0,460.0,156231.0,165.780159,294.435803,58.0
Huelva (capital),10101.0,352.0,742.0,143837.0,244.721456,515.861705,54.0
Sierra de Huelva-Andévalo Central,3145.0,108.0,162.0,67391.0,160.258788,240.388182,22.0
Lepe,2390.0,76.0,150.0,27880.0,272.596844,538.020086,17.0
Isla Cristina,2487.0,74.0,160.0,21393.0,345.907540,747.908194,11.0
Punta Umbría,943.0,43.0,72.0,15355.0,280.039075,468.902638,10.0
Ayamonte,1527.0,69.0,117.0,21104.0,326.952237,554.397271,10.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Cortelazor,17.0,4.0,4.0,299.0,1337.792642,1337.792642,2.0
Castaño del Robledo,9.0,2.0,2.0,228.0,877.192982,877.192982,0.0
Niebla,313.0,8.0,32.0,4116.0,194.363460,777.453839,2.0
Isla Cristina,2487.0,74.0,160.0,21393.0,345.907540,747.908194,11.0
Beas,263.0,21.0,29.0,4341.0,483.759502,668.048837,2.0
Palos de la Frontera,823.0,49.0,77.0,11742.0,417.305399,655.765628,9.0
Zarza-Perrunal (La),39.0,5.0,8.0,1253.0,399.042298,638.467678,0.0
Ayamonte,1527.0,69.0,117.0,21104.0,326.952237,554.397271,10.0
Aljaraque,1323.0,61.0,118.0,21474.0,284.064450,549.501723,8.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Niebla,313.0,8.0,32.0,4116.0,194.363460,777.453839,2.0,0.250000
Villanueva de los Castillejos,169.0,1.0,3.0,2825.0,35.398230,106.194690,0.0,0.333333
Puebla de Guzmán,150.0,2.0,5.0,3092.0,64.683053,161.707633,2.0,0.400000
Jabugo,74.0,3.0,7.0,2260.0,132.743363,309.734513,1.0,0.428571
Cerro de Andévalo (El),167.0,3.0,7.0,2327.0,128.921358,300.816502,1.0,0.428571
Moguer,1546.0,44.0,99.0,21867.0,201.216445,452.737001,7.0,0.444444
Isla Cristina,2487.0,74.0,160.0,21393.0,345.907540,747.908194,11.0,0.462500
Palma del Condado (La),1240.0,15.0,32.0,10801.0,138.876030,296.268864,4.0,0.468750
Huelva (capital),10101.0,352.0,742.0,143837.0,244.721456,515.861705,54.0,0.474394
